<a href="https://colab.research.google.com/github/andcore20/Fifteen_Hundred_Curso_Dados/blob/main/spark_treat_databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importar bibliotecas

In [ ]:
from pyspark.sql.functions import *
from datetime import date

#2. Conectar com o Datalake

In [ ]:
service_credential = dbutils.secrets.get(scope="scope-curso-eng-dados",key="sp-curso-eng-dados")

spark.conf.set("fs.azure.account.auth.type.stgdevcursomar2023.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.stgdevcursomar2023.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.stgdevcursomar2023.dfs.core.windows.net", "bc2050dd-bf22-45f1-aeb0-6c01a7afc9c6")
spark.conf.set("fs.azure.account.oauth2.client.secret.stgdevcursomar2023.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.stgdevcursomar2023.dfs.core.windows.net", "https://login.microsoftonline.com/24f876f1-4233-4600-80f3-4c2c85764058/oauth2/token")

#3. Verificar a conexão

In [ ]:
path_staging = "abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/"
dbutils.fs.ls(path_staging)

Out[85]: [FileInfo(path='abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/Calendario.csv', name='Calendario.csv', size=116943, modificationTime=1685058858000),
 FileInfo(path='abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/Detranmensal.csv', name='Detranmensal.csv', size=3148651, modificationTime=1685058870000),
 FileInfo(path='abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/Detransemanal_01.csv', name='Detransemanal_01.csv', size=13445, modificationTime=1685058881000),
 FileInfo(path='abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/Feriadosnacionais.csv', name='Feriadosnacionais.csv', size=59131, modificationTime=1685058892000),
 FileInfo(path='abfss://fifteen@stgdevcursomar2023.dfs.core.windows.net/coreliano_andre/staging/Integracao.csv', name='Integracao.csv', size=1284502, modificationTime=1685058905000),
 FileInfo(path='abfss://fifteen@stgdevcursomar2023.

#4. Trazer as bases de calendário, feriados nacionais e a de detran mensal

In [ ]:
path_calendario = path_staging + "Calendario.csv"
path_feriados = path_staging + "Feriadosnacionais.csv"
path_detran_m = path_staging + "Detranmensal.csv"

df_calendario = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(path_calendario)
df_feriados = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(path_feriados)
df_detran_m = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(path_detran_m)

display(df_calendario)
display(df_feriados)
display(df_detran_m)

#5. Tratar os dados das tabelas de calendário e feriados nacionais

In [ ]:
#padronizando formato das datas
df_calendario_v2 = df_calendario.withColumn("column1", regexp_replace(col("column1"),"00:00:00.0000000","")).select(col("column1").alias("Data_calendario")) # transformando coluna "column1" em data e renomeando
df_feriados_v2 = df_feriados.withColumn("Data", regexp_replace(col("Data"),"00:00:00.0000000","")) # tirando parte da informação da coluna data


In [ ]:
display(df_calendario_v2)
display(df_feriados_v2)

#6. criar uma coluna na tabela DETRANMENSAL que contenha a data completa e manter colunas DETRANMENSAL: MARCA, MODELO, DATA e VIN

In [ ]:
#juntando colunas dia, mes, ano. E criando uma nova coluna para data
df_detran_m_v2 = df_detran_m.withColumn("Data", concat(col("ano"),lit("-"),col("mes"),lit("-"), col("dia") )).drop("dia").drop("ano").drop("mes")#

#transformando a coluna em tipo data pra ficar mais correta
df_detran_m_v2 = df_detran_m_v2.withColumn("Data", to_date(df_detran_m_v2.Data))# to_date() passa coluna pra tipo data

#tirando coluna versao
df_detran_m_v2 = df_detran_m_v2.drop("versao")

df_detran_m_v2.show()

+----------+----------+-----------------+----------+
|     marca|    modelo|              vin|      Data|
+----------+----------+-----------------+----------+
|   PEUGEOT|      2008|VR3UDYHZSNJ526248|2022-01-06|
| CHEVROLET|      SAIL|LSGHD52H3ND024627|2022-01-06|
|   PEUGEOT|   PARTNER|VR3EF9HPANJ531887|2022-01-04|
| CHEVROLET|   CAPTIVA|LZWADAGA7NB024303|2022-01-24|
| CHEVROLET|  N400 MAX|LZWCDAGA3NC807556|2022-01-03|
|      FORD|NEW RANGER|MPBUR2366NX359311|2022-01-06|
| CHEVROLET|    GROOVE|LZWADAGA8NG006685|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA2NG005841|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA7NG006144|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA1NG007063|2022-01-03|
|   PEUGEOT|      2008|VR3USHNSSNJ525794|2022-01-03|
| CHEVROLET|   CAPTIVA|LZWADAGA6NB027919|2022-01-24|
| CHEVROLET|   CAPTIVA|LZWADAGAXNB027826|2022-01-13|
|   PEUGEOT|       208|VR3UPHMGBNT002023|2022-01-04|
|   PEUGEOT|       208|VR3UPHMGBNT001380|2022-01-06|
|   HYUNDAI|  ATOS AH2|MALAF51CANM163805|2022-

#7. como foi a venda dos carros da marca PEUGEOUT durante os meses

In [ ]:
df_detran_m_v2.count()

Out[90]: 44653

In [ ]:
df_detran_m_v2.dropDuplicates()
df_detran_m_v2.distinct()
df_detran_m_v2.count() # contagem antes e depois para verificar mudança

Out[91]: 44653

In [ ]:
# venda dos carros da marca PEUGEOUT durante os meses
df_detran_m_v2 = df_detran_m_v2.filter("length(VIN) == 17")# filter() pegar so quando placa tem 17 termos
vendas_peugeout = df_detran_m_v2.where(upper(trim(col("marca"))) == "PEUGEOT")# pegando apenas marcas da PEUGEOUT, upper() garante que sempre vai estar maiusculo, trim() tirar espaços da esquerda e direita
vendas_peugeout_mes = vendas_peugeout.select( "marca", month('Data').alias('mes'))# month() pega o mes da coluna data
vendas_peugeout_mes_count = vendas_peugeout_mes.groupBy(vendas_peugeout_mes.mes).count()# groupBy() agrupar por meses, count() vai contar quantidade de linhas pra cada mes
vendas_peugeout_mes_count.sort(asc("mes")).show() # sort(asc()) ordena mes em ordem crescente

+---+-----+
|mes|count|
+---+-----+
|  1| 1499|
|  2| 1194|
|  3|  663|
|  4|  870|
|  5|  571|
|  6|  723|
|  7| 1005|
|  8|  907|
+---+-----+



#8. Os modelos de veículo: GOL e HILUX estão incorretos. Na verdade, deveriam ser respectivamente UP e SW4

In [ ]:
# deixar visual a quantidade dos modelos GOL, HILUX, SW4 e UP
df_detran_m_v3 = df_detran_m_v2
GOL = df_detran_m_v3.where(col("modelo")=="GOL").count()
HILUX = df_detran_m_v3.where(col("modelo")=="HILUX").count()
UP = df_detran_m_v3.where(col("modelo")=="UP").count()
SW4 = df_detran_m_v3.where(col("modelo")=="SW4").count()

modelo_count = [{'GOL': GOL, 'HILUX': HILUX,'UP':UP,'SW4':SW4}]
display(modelo_count)

GOL,HILUX,SW4,UP
947,2264,0,0


In [ ]:
df_detran_m_v3 = df_detran_m_v3.withColumn("modelo",regexp_replace(col("modelo"),"GOL","UP")) # trocando "GOl" por "UP"
df_detran_m_v3 = df_detran_m_v3.withColumn("modelo",regexp_replace(col("modelo"),"HILUX","SW4")) # trocando "HILUZ" por "SW4"


In [ ]:
# verificando que mudou certo
GOL = df_detran_m_v3.where(col("modelo")=="GOL").count()
HILUX = df_detran_m_v3.where(col("modelo")=="HILUX").count()
UP = df_detran_m_v3.where(col("modelo")=="UP").count()
SW4 = df_detran_m_v3.where(col("modelo")=="SW4").count()

modelo_count = [{'GOL': GOL, 'HILUX': HILUX,'UP':UP,'SW4':SW4}]
display(modelo_count)

GOL,HILUX,SW4,UP
0,0,2264,947


#9. Não consider os feriados de CARNAVAL, INDEPÊNDENCIA DO BRASIL e NOSSA SENHORA APARECIDA.

In [ ]:
# identificando nome completo da coluna Feriado
valores_feriado = df_feriados_v2.select(col("Feriado")).distinct().collect()
for valor in valores_feriado:
    print(valor[0])

Tiradentes
Proclamação da República
Nossa Sr.a Aparecida - Padroeira do Brasil
Dia do Trabalho
Corpus Christi
Natal
Confraternização Universal
Carnaval
Independência do Brasil
Paixão de Cristo
Finados


In [ ]:
df_feriados_v2.count()# quantidade de linhas antes

Out[97]: 936

In [ ]:
# filtrando para aparecer apenas quado o df é diferente dos feriados especificados
df_feriados_v3 = df_feriados_v2.where((col("Feriado") != "Carnaval") &
                                     (col("Feriado") != "Independência do Brasil") &
                                     (col("Feriado") != "Nossa Sr.a Aparecida - Padroeira do Brasil"))
df_feriados_v3.count()# quantidade de linhas depois

Out[98]: 624

#10. Fazer um join da base de detran com a base de calendário (final mostrar Data_calendario)

##10.1 Resolvendo

In [ ]:
df_detran_m_v3.count()# quantidade de linhas em detran antes do join

Out[99]: 44602

In [ ]:
# df_detran_m_v3 (esquerda), df_calendario_v2 (direita)
# como queria manter todo conteudo de Data_calendario usei join right

join_detran_calendario = df_detran_m_v3.join(df_calendario_v2, df_detran_m_v3.Data == df_calendario_v2.Data_calendario, "right")\
    .drop(df_detran_m_v3.Data)\
    .where((col("Data_calendario") > "2022-01-01"))# tirar datas anteriores porque estavam como null


display(join_detran_calendario)


In [ ]:
join_detran_calendario.count()# quantidade de linhas em detran depois do join

Out[101]: 45471

##10.2 Resolvendo com formato do SQL

In [ ]:
# criando TempView para DF utilizados
df_detran_m_v3.createOrReplaceTempView("detran_mensal")
df_calendario_v2.createOrReplaceTempView("calendario")

In [ ]:

teste2 = spark.sql(
    """
    SELECT *
    FROM detran_mensal A
    RIGHT JOIN calendario B
    ON A.Data =  B.Data_calendario
    WHERE B.Data_calendario > "2022-01-01"
    """)
# WHERE para tirar datas anteriores porque estavam como null
display(teste2)
teste2.count()

#11. Fazer um join da base resultante do exercício anterior com a base de feriados.

In [ ]:
join_dc_feriados = join_detran_calendario.join(df_feriados_v3, join_detran_calendario.Data_calendario == df_feriados_v3.Data,"left").drop("Data").where((col("Data_calendario") >= "2022-01-01"))

display(join_dc_feriados)

In [ ]:
join_dc_feriados.count()

Out[105]: 45471

In [ ]:
quantidade_feriados_join = join_dc_feriados.where(col("Feriado") != "null").count()
quantidade_feriados = df_feriados_v3.count()

In [ ]:
display(quantidade_feriados_join)

443

In [ ]:
display(quantidade_feriados)

624